In [ ]:
from pathlib import Path
import zarr
import dask.array as da
import os
import imageio.v2 as imageio
import sys
from zarr import parse_url
from ome_zarr.io import Reader
from dask import delayed

ome_zarr_dir = '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/DK184/www/neuroglancer_data/'
INPUT_dye = Path(ome_zarr_dir, 'C1.zarr')
INPUT_virus = Path(ome_zarr_dir, 'C3.zarr')
input_format = "ome-zarr"
str_section_number = "001"  # Example section number (zfill for legacy processing)


def load_image(file: str):
    if os.path.exists(file):
        return imageio.imread(file)
    else:
        print(f'ERROR: {file} NOT FOUND')
        sys.exit(1)

# Process Virus Data
store_virus = parse_url(INPUT_virus, mode="r").store
zarr_root_virus = parse_url(INPUT_virus, mode="r").root
img_stack_virus = zarr_root_virus["scale0"]  # Access full-resolution data

# Convert the Zarr array to a Dask array for lazy loading
dask_data_virus = da.from_zarr(img_stack_virus)
total_sections_virus = dask_data_virus.shape[1]  # Shape is (c, z, y, x)

# Loop through each section (z-axis) for Virus data
input_file_virus = []
for z in range(total_sections_virus):
    # Extract the x, y slice for the current z-layer (assuming channel 0)
    section_data_virus = dask_data_virus[0, z, :, :]  # Lazy loading
    input_file_virus.append(section_data_virus)

# Process Dye Data
store_dye = parse_url(INPUT_dye, mode="r").store
zarr_root_dye = parse_url(INPUT_dye, mode="r").root
img_stack_dye = zarr_root_dye["scale0"]  # Access full-resolution data

dask_data_dye = da.from_zarr(img_stack_dye)
total_sections_dye = dask_data_dye.shape[1]  # Shape is (c, z, y, x)

input_file_dye = []
for z in range(total_sections_dye):
    # Extract the x, y slice for the current z-layer (assuming channel 0)
    section_data_dye = dask_data_dye[0, z, :, :]  # Lazy loading
    input_file_dye.append(section_data_dye)

# Create delayed tasks for processing the sections (SEPARATE TASK LIST PER CHANNEL)
delayed_tasks_virus = [delayed(load_image)(section) for section in input_file_virus]
delayed_tasks_dye = [delayed(load_image)(section) for section in input_file_dye]


store_dye.close()
store_virus.close()

Dimensions of the array (c, z, y, x): (1, 348, 38835, 77383)
[{'name': 'c', 'type': 'channel'}, {'name': 'z', 'type': 'space', 'unit': 'micrometer'}, {'name': 'y', 'type': 'space', 'unit': 'micrometer'}, {'name': 'x', 'type': 'space', 'unit': 'micrometer'}]


'\n# Convert the zarr array to a dask array for efficient chunked reading\n#dask_array = da.from_zarr(dye_img_stack)\n\n# Get the number of layers (z-axis)\nnum_layers = dask_array.shape[0]\n\n# Loop through each layer along the z-axis\nfor z in range(num_layers):\n    # Read the x, y slice for the current z-layer\n    xy_slice = dask_array[z, :, :].compute()  # .compute() loads the data into memory\n    \n    # Process or analyze the xy_slice as needed\n    print(f"Layer {z}:")\n    print(xy_slice)\n\n\n# Example: Accessing a specific resolution level\n# resolution_level = 0  # 0 is typically the highest resolution\n# array_3d = root[\'data\'][resolution_level]\n\n# store.close()\n'

In [ ]:
def zarr_to_dask(INPUT: Path):
    '''
    create dask task arrays (delayed loading) for each Zarr image stack
    '''

    # Process Virus Data
    store = parse_url(INPUT, mode="r").store
    zarr_root = parse_url(INPUT, mode="r").root
    img_stack = zarr_root["scale0"]  # Access full-resolution data

    # Convert the Zarr array to a Dask array for lazy loading
    dask_data = da.from_zarr(img_stack)
    total_sections = dask_data.shape[1]  # Shape is (c, z, y, x)

    # Loop through each section (z-axis) for Virus data
    input_filelist = []
    for z in range(total_sections):
        # Extract the x, y slice for the current z-layer (assuming channel 0)
        total_sections = dask_data_virus[0, z, :, :]  # Lazy loading
        input_filelist.append(total_sections)
        
    store.close()

    # Create delayed tasks for processing the sections (SEPARATE TASK LIST PER CHANNEL)
    return [delayed(load_image)(section) for section in input_filelist]


ome_zarr_dir = '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/DK184/www/neuroglancer_data/'
INPUT_dye = Path(ome_zarr_dir, 'C1.zarr')
INPUT_virus = Path(ome_zarr_dir, 'C3.zarr')
input_format = "ome-zarr"
str_section_number = "001"  # Example section number (zfill for legacy processing)

delayed_tasks_dye = zarr_to_dask(INPUT_dye)
delayed_tasks_virus = zarr_to_dask(INPUT_virus)